In [3]:
import pandas as pd
import nltk
import gensim

In [112]:
import Levenshtein

In [4]:
reviews_df = pd.read_csv("metacritic_critic_reviews.csv")

In [110]:
steam_data_df = pd.read_csv("steam-store-games/steam.csv")

In [115]:
steam_data_df

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19


In [135]:
(cut_df.groupby("game").count()).index

Index(['A Game of Thrones: Genesis', 'A Hat in Time',
       'A Total War Saga: Thrones of Britannia', 'ADR1FT', 'Absolver',
       'Act of Aggression', 'Age of Empires Online',
       'Age of Empires: Definitive Edition', 'Age of Wonders III', 'Agony',
       ...
       'World of Warplanes', 'World of Warships', 'Wreckfest', 'XCOM 2',
       'XCOM 2: War of the Chosen', 'XCOM: Enemy Unknown',
       'XCOM: Enemy Within', 'Yesterday', 'Yooka-Laylee', 'Zeno Clash II'],
      dtype='object', name='game', length=483)

In [137]:
steam_game_names = [x.lower() for x in steam_data_df["name"]]
meta_critic_names = [str(x).lower() for x in cut_df.groupby("game").count().index]

In [144]:
steam_game_genres = {}
for index, data in steam_data_df.iterrows():
    steam_game_genres[data["name"].lower()] = data["genres"]

In [167]:
genre_dict = {}
for x in meta_critic_names:
    for y in steam_game_names:
        if Levenshtein.distance(x, y) < 3:
            genre_dict[x] = steam_game_genres[y]
            break
        else:
            genre_dict[x] = 0

In [168]:
len(genre_dict)

483

In [191]:
for x in steam_game_names:
    for y in x.split(":"):
        if Levenshtein.distance(y, "sid meier's civilization vi") < 3:
            print(x)

sid meier's civilization® v
sid meier's civilization iv: colonization
sid meier’s civilization® vi


In [170]:
for x in genre_dict:
    if genre_dict[x] == 0:
        truncated_title = x.split(":")[0]
        for y in steam_game_names:
            if Levenshtein.distance(truncated_title, y) < 2:
                genre_dict[x] = steam_game_genres[y]

In [171]:
for x in genre_dict:
    if genre_dict[x] == 0:
        truncated_title = x.split(":")[0]
        for y in steam_game_names:
            truncated_steam_title = y.split(":")[0]
            if Levenshtein.distance(truncated_title, truncated_steam_title) < 2:
                genre_dict[x] = steam_game_genres[y]

In [194]:
for x in genre_dict:
    if genre_dict[x] == 0:
        truncated_title = x.split(":")[0]
        for y in steam_game_names:
            for z in y.split(":"):
                if Levenshtein.distance(truncated_title, z) < 3:
                    print(x,y)

alice: madness returns splice
alice: madness returns alien: isolation
alice: madness returns slide: platformer
alice: madness returns price
alice: madness returns live
alice: madness returns alone
alice: madness returns alone
alice: madness returns algae
battlefield 1 battlefield: bad company™ 2
battlefield 3 battlefield: bad company™ 2
battlefield 4 battlefield: bad company™ 2
battlefield v battlefield: bad company™ 2
broken sword 5: the serpents' curse - part i broken sword: director's cut
resident evil 4: ultimate hd edition resident evil: operation raccoon city
sid meier's civilization vi: rise and fall sid meier's civilization® v
sid meier's civilization vi: rise and fall sid meier's civilization iv: colonization
sid meier's civilization vi: rise and fall sid meier’s civilization® vi
simcity civcity: rome
state of decay 2 state of decay: yose
the raven: legacy of a master thief the cave
the raven: legacy of a master thief the cavern
the raven: legacy of a master thief the tavern
t

In [185]:
from collections import defaultdict

In [172]:
for x in genre_dict:
    if genre_dict[x] == 0:
        print(x)

act of aggression
age of empires online
age of empires: definitive edition
alice: madness returns
among the sleep
battlefield 1
battlefield 3
battlefield 4
battlefield hardline
battlefield v
bombshell (2016)
broken sword 5: the serpents' curse - part i
destiny 2
diablo iii
diablo iii: reaper of souls
don't starve
dragon age ii
dreamfall chapters book one: reborn
dustforce
endless space
fable iii
gauntlet (2014)
guild wars 2
hearthstone: heroes of warcraft
heroes of the storm
jurassic park: the game
kentucky route zero - act i
lawbreakers
leisure suit larry: reloaded
marvel heroes
metro redux
never alone
overkill's the walking dead
offworld trading company
overwatch
resident evil 4: ultimate hd edition
resident evil hd remaster
sea of thieves
sid meier's civilization vi: rise and fall
simcity
sniper elite iii
star wars battlefront ii
starcraft ii: heart of the swarm
starcraft ii: legacy of the void
starcraft remastered
state of decay 2
sword coast legends
the raven: legacy of a master t

In [189]:
genre_count

defaultdict(int,
            {'Action': 11903,
             'Free to Play': 1704,
             'Strategy': 5247,
             'Adventure': 10032,
             'Indie': 19421,
             'RPG': 4311,
             'Animation & Modeling': 79,
             'Video Production': 38,
             'Casual': 10210,
             'Simulation': 5194,
             'Racing': 1024,
             'Violent': 843,
             'Massively Multiplayer': 723,
             'Nudity': 266,
             'Sports': 1322,
             'Early Access': 2954,
             'Gore': 537,
             'Utilities': 146,
             'Design & Illustration': 87,
             'Web Publishing': 28,
             'Education': 51,
             'Software Training': 31,
             'Sexual Content': 245,
             'Audio Production': 29,
             'Game Development': 17,
             'Photo Editing': 12,
             'Accounting': 6,
             'Documentary': 1,
             'Tutorial': 1})

In [188]:
genre_count = defaultdict(int)
for x in steam_data_df["genres"]:
    for y in x.split(";"):
        genre_count[y.strip()] += 1

In [5]:
reviews_df.head()

,name,review,game,platform,score,date
0,LEVEL (Czech Republic),"Portal 2 is a masterpiece, a work of art that ...",Portal 2,PC,100.0,"May 25, 2011"
1,GameCritics,So do we need Portal 2? Do I need it? Maybe no...,Portal 2,PC,100.0,"May 8, 2011"
2,PC Games (Russia),Portal 2 exceeds every expectation. It has a s...,Portal 2,PC,100.0,"May 6, 2011"
3,Adventure Gamers,"Like its predecessor, Portal 2 is not an adven...",Portal 2,PC,100.0,"Apr 29, 2011"
4,Armchair Empire,"Pile on the ""Oh, yes!"" moments of solving some...",Portal 2,PC,100.0,"Apr 28, 2011"


In [94]:
selector_array = []
for x in cut_df["game"]:
    if x in major_games:
        selector_array.append(True)
    else:
        selector_array.append(False)

In [88]:
focus_df = reviews_df[reviews_df["platform"] == "PC"]

In [95]:
cut_df = cut_df[selector_array]

In [97]:
cut_df

,name,review,game,platform,score,date
0,LEVEL (Czech Republic),"Portal 2 is a masterpiece, a work of art that ...",Portal 2,PC,100.0,"May 25, 2011"
1,GameCritics,So do we need Portal 2? Do I need it? Maybe no...,Portal 2,PC,100.0,"May 8, 2011"
2,PC Games (Russia),Portal 2 exceeds every expectation. It has a s...,Portal 2,PC,100.0,"May 6, 2011"
3,Adventure Gamers,"Like its predecessor, Portal 2 is not an adven...",Portal 2,PC,100.0,"Apr 29, 2011"
4,Armchair Empire,"Pile on the ""Oh, yes!"" moments of solving some...",Portal 2,PC,100.0,"Apr 28, 2011"
...,...,...,...,...,...,...
125463,Worth Playing,"In a way, the experience you'll have with the ...",Agony,PC,30.0,"Jun 1, 2018"
125464,GameSpot,I entered Agony’s Gates of Hell with a slack-j...,Agony,PC,30.0,"May 31, 2018"
125465,Twinfinite,The only people who should consider Agony are ...,Agony,PC,30.0,"May 29, 2018"
125466,The Overpowered Noobs,Agony is a survival horror title that serves a...,Agony,PC,20.0,"Jun 14, 2018"


In [109]:
pd.Series([len(x) for x in cut_df["review"]]).quantile(.25)

142.0

In [98]:
print(major_games)

['A Game of Thrones: Genesis', 'A Hat in Time', 'A Total War Saga: Thrones of Britannia', 'ADR1FT', 'Absolver', 'Act of Aggression', 'Age of Empires Online', 'Age of Empires: Definitive Edition', 'Age of Wonders III', 'Agony', 'Alan Wake', 'Alice: Madness Returns', 'Alien: Isolation', 'Aliens: Colonial Marines', 'American Truck Simulator', 'Amnesia: A Machine for Pigs', 'Among the Sleep', 'Anno 2070', 'Anno 2205', 'Anomaly: Warzone Earth', 'Antichamber', 'Apotheon', 'Arma 3', 'Armikrog', "Assassin's Creed Origins", "Baldur's Gate II: Enhanced Edition", "Baldur's Gate: Enhanced Edition", 'Banished', 'Batman: The Enemy Within - Episode 1: The Enigma', 'Batman: The Telltale Series - Episode 1: Realm of Shadows', 'BattleTech', 'Battleborn', 'Battlefield 1', 'Battlefield 3', 'Battlefield 4', 'Battlefield Hardline', 'Battlefield V', 'Battlefleet Gothic: Armada', 'Betrayer', 'BioShock Infinite', 'BioShock Infinite: Burial at Sea - Episode One', 'BioShock Infinite: Burial at Sea - Episode Two'

In [93]:
major_games = []
for game, counts in focus_df.groupby("game").count().iterrows():
    if counts[0] > 27:
        major_games.append(game)

In [68]:
focus_df.groupby("game").count()

,name,review,platform,score,date
game,,,,,
.hack//G.U. Last Recode,7,7,7,7,7
0RBITALIS,9,9,9,9,9
10 Second Ninja,13,13,13,13,13
11-11: Memories Retold,11,11,11,11,11
1954: Alcatraz,24,24,24,24,24
...,...,...,...,...,...
Zombie Night Terror,21,21,21,21,21
Zwei: The Ilvard Insurrection,8,8,8,8,8
continue?9876543210,15,15,15,15,15


In [74]:
focus_df.groupby("game").count()["review"].median()


15.0

In [45]:
from gensim.models.ldamulticore import LdaMulticore

In [7]:
from nltk import sent_tokenize

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/victor/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
simple_preprocess()

In [29]:
preprocessed_text = []
for x in reviews_df["review"]:
    preprocessed_text.append(simple_preprocess(x))

In [36]:
preprocessed_text_2 = []
for x in preprocessed_text:
    rm_stopwords = []
    for y in x:
        if y not in STOPWORDS:
            rm_stopwords.append(y)
    preprocessed_text_2.append(rm_stopwords)

In [37]:
corpa_dict = Dictionary(preprocessed_text_2)

In [38]:
bow_corpa = []
for x in preprocessed_text_2:
    bow_corpa.append(corpa_dict.doc2bow(x))

In [39]:
bow_corpa[:10]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1)],
 [(4, 1),
  (13, 4),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1)],
 [(13, 1),
  (14, 1),
  (16, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1)],
 [(4, 1),
  (13, 1),
  (16, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1)],
 [(4, 1),
  (14, 1),
  (18, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1)],
 [(4, 1),
  (13, 2),
  (27, 1),
  (52, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1

In [21]:
from gensim.parsing.preprocessing import STOPWORDS

In [26]:
from gensim.corpora import Dictionary

In [22]:
from gensim.utils import simple_preprocess

In [53]:
lda_model = LdaMulticore(bow_corpa, id2word=corpa_dict, passes =10, num_topics = 20)

In [54]:
lda_model.print_topics()

[(0,
  '0.031*"game" + 0.025*"story" + 0.020*"world" + 0.014*"experience" + 0.013*"adventure" + 0.013*"gameplay" + 0.009*"narrative" + 0.009*"puzzles" + 0.009*"interesting" + 0.008*"beautiful"'),
 (1,
  '0.026*"game" + 0.025*"new" + 0.019*"vr" + 0.018*"series" + 0.015*"games" + 0.014*"fans" + 0.011*"man" + 0.011*"franchise" + 0.010*"best" + 0.009*"great"'),
 (2,
  '0.024*"game" + 0.019*"series" + 0.017*"best" + 0.014*"games" + 0.014*"years" + 0.013*"evil" + 0.012*"new" + 0.011*"fans" + 0.011*"pokémon" + 0.009*"park"'),
 (3,
  '0.025*"game" + 0.009*"like" + 0.008*"world" + 0.005*"human" + 0.005*"experience" + 0.005*"players" + 0.005*"games" + 0.005*"revolution" + 0.004*"new" + 0.004*"rally"'),
 (4,
  '0.047*"game" + 0.011*"gameplay" + 0.010*"genre" + 0.010*"interesting" + 0.010*"strategy" + 0.010*"good" + 0.010*"fun" + 0.009*"combat" + 0.008*"experience" + 0.008*"rpg"'),
 (5,
  '0.051*"old" + 0.021*"lego" + 0.018*"star" + 0.017*"school" + 0.015*"game" + 0.014*"games" + 0.012*"wars" + 0.